This notebook contains several basic functions that is useful in atmospheric sciences research 

# atmospheric physics 

In [2]:
'''
function of calculating saturated water vapor mixing ratio

input: 
temp: an array of temperature
pr: an array or a number of corresponding temperature

output: 
h2o: water vapor mixing ratio in ppmv (part per million)

Wandi Yu
10.12.2015
'''
import numpy as np
def SAT(temp,pr): 
    esi=np.zeros(np.shape(temp),dtype='f')
    i_h2o=np.where(temp>273.15)
    n_h2o=np.size(i_h2o)
    if n_h2o>0:
        esi[i_h2o]=np.exp(54.842763-6763.22/temp[i_h2o]-4.210*np.log(temp[i_h2o])\
        +0.000367*temp[i_h2o]+np.tanh(0.0415*(temp[i_h2o]-218.8))\
        *(53.878-1331.22/temp[i_h2o])-9.44523*np.log(temp[i_h2o]+0.014025*temp[i_h2o]))/100.
    i_ice=np.where((temp>0.0) & (temp<273.15))
    n_ice=np.size(i_ice)
    if n_ice>0:
        esi[i_ice]=np.exp(9.550426-5723.265/temp[i_ice]+3.53068*np.log(temp[i_ice])\
        -0.00728332*temp[i_ice])/100.0
    h2o=esi/(pr-esi)
    high=np.where((esi*100-pr)<0)
    nh=np.size(high)
    if nh>0:
        h2o[high]=(esi/pr)[high]
    return h2o*1.0e6

In [3]:
'''
function to calculate potential temperature
input: 
    t: temperature
    p: pressure
    p0: surface pressure, default 1000
output: 
    theta: potential temperature
'''
import numpy as np
def cal_theta(t,p,p0=1000):
    return t*(p0/p)**0.286

In [4]:
'''
function to convert mid altitude height to pressure 
    !!!! pay attention: only for middle altitude: 10-20km
    input:
        z: height in km
    output:
        p: pressure in hPa
'''
import numpy as np
def z2p(z):
    return 1000*np.exp(-z/7)

In [6]:
'''
function to convert mid altitude presure to height
    !!!! pay attention: only for middle altitude: 10-20km
    input:
        p: pressure in km
    output:
        z: height in hPa
'''
def z2p(p):
    return -7*np.log(fback.Lev.values/1000)

# statistics

In [11]:
'''
using the monte carlo method to calculate the p-value for the null hypothesis: sample1>sample2

input: 
temp: an array of temperature
pr: an array or a number of corresponding temperature

output: 
h2o: water vapor mixing ratio in ppmv (part per million)

Wandi Yu
10.12.2015
'''
import numpy as np
def monte_carlo_comparison(sample1, sample2):
    N=10000
    sample1 = sample1[np.isfinite(sample1)]
    sample2 = sample2[np.isfinite(sample2)]
    sample = np.append(sample1,sample2)
    s1 = np.size(sample1)
    xbar = np.ones(N)
    for i in range (N):
        np.random.shuffle(sample)
        xbar[i] = np.nanmean(sample[:s1])-np.nanmean(sample[s1:])
    p = len(np.where(xbar>=(sample1.mean()-sample2.mean()))[0])/float(N)
    return p

In [16]:
'''
a significance test for the correlation coefficient between array a and b, print out the p value

input: 
a, b: two arrays with the same length
sig_level: significance level 
auto_correlation: whether consider the auto_correlation 

output:
p: p-value 

Wandi Yu
10.07.2019
'''
from scipy import stats
import numpy as np
def r_significance_test(a,b,sig_level=0.975,auto_correlation = True):
    sig_level = (1-sig_level)/2.       # two-tailed
    if auto_correlation:
        aa,bb = np.polyfit(a,b,1)
        residual = (b-aa*a-bb)
        x = anml(residual)
        z = np.correlate(x,x,mode='full')
        z = z[len(z)//2:]
        lag1 = z[1]/z.max()
        df = (len(a)*(1-lag1)/(1+lag1))-2
    else:
        df = len(a)
    r = np.corrcoef(a,b)[0,1]
    t = r*np.sqrt(df)/np.sqrt(1-r**2)
    if t>stats.t.ppf(sig_level,df):
        print ('Significance test passed')
    else:
        print  ('not significant')
    p = stats.t.pdf(t,df)
    return p

In [7]:
'''
calculate the multivariate regression
input: 
    x: the independent variable 
    y: the dependent variable
output:
    intercept, coefficient, and prediction values
04.14.2016
'''
class least_square():
    def __init__(self):
        self.X = 0             # dependent variable
        self.y = 0             # independent variable
        self.intercept = 0     # intercept of the fit
        self.coef = 0          # coefficent of the fit 
        self.predict = 0       # the predict value of the fit 
    def fit(self,A,y):   
    # find out the linear fit for A and y
    # A: independent variable, first dimension must be the same as y
    # y: dependent variable
        self.X = A
        self.y = y
        Y = self.y[:,np.newaxis]
        out = np.squeeze(np.dot(np.linalg.inv(np.dot(A.T,A)),np.dot(A.T,Y)))
        self.intercept, self.coef = out[0],out[1:]
        self.predict = np.dot(A,np.append(f.intercept,f.coef))

# time-series and geophysics

In [19]:
'''
calculate the anomaly regarding to monthly mean

input: 
x: xarray data 

output:
anom: anomaly of the data

Wandi Yu
03.07.2018
'''
def anml(x):
    clim = x.groupby('time.month').mean(dim='time')
    anom = x.groupby('time.month')-clim
    return anom


In [20]:
'''
calculate the anomaly regarding to monthly mean

input: 
x: 1-d array
n: number of data per month

output:
anom: anomaly of the data

Wandi Yu
03.07.2018
'''
def anml(x):
    l = len(x)
    mean = np.nanmean(np.nanmean(x.reshape([l//12//n,12,n]),axis=0),axis=1)
    return (x.reshape([l//12//n,12,n])-mean[np.newaxis,:,np.newaxis]).reshape(l)
    return anom


In [12]:
'''
given start year and end year, return a list of string representing each month
'''
def year2mon(startyear,endyear):
    years = np.arange(startyear,endyear,dtype='int')
    return [str(year).zfill(4)+str(month+1).zfill(2) for year in years for month in range(12)]

In [14]:
'''
input: a 1-d array 
output: the detrended array
'''
def detrend(arr):
    s=len(arr)
    year=np.arange(s)
    a,b = np.polyfit(s,arr,1)    
    detrend=arr-a-s*b
    return detrend

In [ ]:
'''
calculate the seasonal cycle of a variable 
input: 
    var: numpy array 
output: 
    var_new: the seasonal cycle of the variable 
'''
def seasonal_cycle(var):
    s=var.shape
    if len(s)==1:
        var_new=var.reshape(s[0]/12,12)
    else:
        var_new=var.reshape(s[0]/12,12,s[1:])
    return np.nanmean(var_new,axis=0)

In [21]:
'''
a function to calculate weighted area mean globally or tropically, 
   considering the weighted average regrading to latitude
 input: 
   var: numpy array, the last two dimenstion is lat and lon
   lat: latitude
   tropical: default 0, global mean
           tropical=1: calculate tropical mean.
 output:

   10.06.2016
   Wandi Yu
'''

def areaweightmean(var,lat,tropical=0):
    vard=len(var.shape)
    if tropical==1:
        i_tropical=np.all([lat>-30,lat<30],axis=0)
        lat=lat[i_tropical]
        if vard==2:
            var=var[i_tropical]
        elif vard==3:
            var=var[:,i_tropical]
        elif vard==4:
            var=var[:,:,i_tropical]
        else:
            raise ValueError('error in variable dimension')
    weight=np.cos(np.radians(lat))
    lonmean=np.nanmean(var,axis=-1)
    latsum=np.nansum(lonmean*weight[np.newaxis,np.newaxis,:],axis=-1)
    weightsum=np.nansum(weight)
    return latsum/weightsum

In [15]:
'''
calculate the gradient of a variable, in meridianal, zonal, or vertical direction 
input: 
    var: a 4-d array, dimension: time,pressure,longitude,latitude
    pressure: the pressure axis
    lon: longitude axis
    lat: latitude axis
    kw: option of calculating the gradient, meridianal, zonal, or vertical
04.05.2017
Wandi Yu
'''
def gradient(var,pressure,lon,lat,kw):
    r_earth=6371000.
    dlambda=np.radians(lon[1]-lon[0])
    dphi=np.radians(lat[1]-lat[0])
    if kw=='dx':
        return ((var[:,:,:,1:]-var[:,:,:,:-1])/(dlambda*r_earth*
        np.cos(np.radians(lat)))[np.newaxis,np.newaxis,:,np.newaxis])[:,:-1,:-1,:]
    if kw=='dy':
        return (var[:,:,1:,:]-var[:,:,:-1,:])[:,:-1,:,:-1]/(dphi*r_earth)
    if kw=='dz':
        return (var[:,1:]-var[:,:-1])[:,:,:-1,:-1]/(pressure[:,1:]\
        -pressure[:,:-1])[:,:,:-1,:-1]

In [16]:
'''
interp between differnt pressure field 
input:
    pressure: old pressure field
    mix: the variable
    prgrid: new pressure field
ouput: 
    pr_mix: variable in new pressure grid
'''
def interp_pre(pressure,mix,prgrid):
    s=pressure.shape
    pr_mix=np.zeros([s[0],len(prgrid),s[2],s[3]])
    for i in range(s[0]):
        for j in range(s[2]):
            for k in range(s[3]):
                pr_mix[i,:,j,k] =np.interp(np.log(prgrid),\
                np.log(pressure[i,:,j,k]),mix[i,:,j,k])
    return pr_mix